In [1]:
from sklearn.cluster import KMeans
import numpy as np
import scipy as sp
import sys
sys.path.append('../corv/src')
sys.path.append('../wdtools')

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm
import pandas as pd
import corv

### Query
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c
import data_selector as ds

import wdtools
import corv


using local paths
star and exposure catalogs not found! check paths and run make_catalogs() if you want to use sdss functionality. otherwise ignore.


2022-07-14 16:40:03.770668: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-14 16:40:03.770759: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Please install galpy if you want to use the gaia module. otherwise, ignore this.


In [2]:
d1 = pd.read_csv('data/wd_spectra.csv')
d1 = Table.from_pandas(d1)

catalog = Table.read('data/00_raw.fits')
print(len(catalog))

2000


In [ ]:
spAll = Table.read('~/Downloads/spAll-master.fits')

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table, join_skycoord
from astropy import table

print(spAll.keys())

print(spAll[['RACAT', 'DECCAT']])

spAll = spAll[spAll['DECCAT'] < 90]
spAll = spAll[spAll['DECCAT'] > -90]

catalog['wd_pos'] = SkyCoord(catalog['wd_ra'], catalog['wd_dec'], unit='deg')
spAll['wd_pos'] = SkyCoord(spAll['RACAT'], spAll['DECCAT'], unit='deg')

join_func = table.join_skycoord(5 * u.arcsecond)
sdss5_catalog = table.join(catalog, spAll, join_funcs={'wd_pos': join_skycoord(5 * u.arcsec)})

In [ ]:
print(sdss5_catalog[['wd_pos_id', 'wd_pos_1', 'wd_pos_2']])

In [ ]:
ADQL_CODE1 = "SELECT \
        sdss.original_ext_source_id as bestobjid,\
        gaia_source.source_id\
        FROM gaiaedr3.gaia_source \
        JOIN gaiaedr3.sdssdr13_best_neighbour as sdss\
        ON gaia_source.source_id = sdss.source_id      \
        WHERE gaia_source.source_id IN {}\
    ".format(tuple(catalog['wd_source_id']))

job1 = Gaia.launch_job(ADQL_CODE1,dump_to_file=False)
d1 = job1.get_results()
print(len(d1))

In [ ]:
drops = []
bestobjid1 = []

for i in tqdm (range(len(catalog))):
    notfound = False
    a = np.where(d1['source_id'] == catalog['wd_source_id'][i])
    
    try:
        j = a[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            bestobjid1.append(d1['bestobjid'][j])
        except:
            notfound = True
    if notfound:
        drops.append(i)    
        
catalog.remove_rows(drops)
catalog['wd_bestobjid'] = bestobjid1

In [ ]:
print(catalog['wd_bestobjid'])

In [ ]:
from astropy.table import Table, vstack, hstack

stardats = []
iters = len(catalog) // 100

for i in tqdm(range(iters)):
    SDSS_QUERY = """select bestObjID, plate, mjd, fiberID
        from dbo.SpecObjAll
        where bestObjID in {}""".format(tuple(catalog['wd_bestobjid'][100*i:100*i+100]))
    
    stardats.append(SDSS.query_sql(SDSS_QUERY))
spec = vstack(stardats)

In [ ]:
print(spec)


In [ ]:
drops = []
plate = []
mjd = []
fiber = []

for i in tqdm (range(len(catalog))):
    notfound = False
    a = np.where(spec['bestObjID'] == catalog['wd_bestobjid'][i])
    
    try:
        j = a[0][0]
    except:
        notfound = True
        
    if not notfound: 
        try:
            
            plate.append(spec['plate'][j])
            mjd.append(spec['mjd'][j])
            fiber.append(spec['fiberID'][j])
            
            
        except:
            notfound = True
    if notfound:
        drops.append(i)    
        
catalog.remove_rows(drops)
catalog['plate'] = plate
catalog['mjd'] = mjd
catalog['fiber'] = fiber

In [ ]:
print(len(catalog))

In [ ]:
spec = []
wl = []
fl = []
ivar = []
drops = []
worked = []

for i in tqdm(range(len(catalog))):
    try:
        spec = SDSS.get_spectra_async(plate=plate[i], fiberID=fiber[i], mjd=mjd[i])[0].get_fits()
        
        #print(spec[4].header)
        
        wl.append(10**spec[1].data['loglam'])
        fl.append(spec[1].data['flux'])
        ivar.append(spec[1].data['ivar'])
        worked.append(i)
    except:
        drops.append(i)
        
print(len(wl))
print(len(fl))
print(len(ivar))
        
catalog.remove_rows(drops)
catalog['wd_wl'] = wl
catalog['wd_fl'] = fl
catalog['wd_ivar'] = ivar

In [ ]:
print(len(catalog))

In [ ]:
def pseudonormalize(fl, win = 50):
    cont_fl = np.repeat(np.nan, len(fl))
    
    for i in range(len(fl)):
        if (i - (win / 2)) < 0:
            fl_win = fl[0:(i + (win // 2))]
        else:
            fl_win = fl[(i - (win // 2)) : (i + (win // 2))]
        
        cont_fl[i] = np.percentile(fl_win, 90)
        
    return fl / cont_fl
'''
def spec_split(bands, wl, fl, window_size = 100):
    spec_split = {}
    
    for i in range(len(bands)):
        spec_split[bands.keys()[i]] = np.array([ [wl[
'''

In [ ]:
i = 1

plt.figure(figsize=(20,8))

plt.plot(catalog['wd_wl'][i], catalog['wd_fl'][i])
plt.grid()
plt.ylabel(r'Flux [$10^{-17}$ ergs/s/cm2/A]')
plt.xlabel(r'Wavelength [A]')
plt.title('White Dwarf Spectrum')
ax = plt.gca()

plt.show()

In [ ]:
print(i)
nwl = catalog['wd_wl'][i]
nfl = catalog['wd_fl'][i]
cont = pseudonormalize(nfl, 50)

plt.figure(figsize=(20,8))

plt.plot(nwl, cont)
plt.grid()
plt.ylabel(r'Normalized Flux')
plt.xlabel(r'Wavelength [A]')
plt.title('Normalized Spectrum')
ax = plt.gca()

plt.show()

In [ ]:
print([np.any(np.isnan(catalog['wd_ivar'][i])) for i in range(len(catalog))])

In [ ]:
from lmfit import Parameters, fit_report, minimize

rvs = []
e_rvs = []
redchi = []
drops = []

#catalog['wd_ivar'] = [catalog['wd_ivar'][i]  for i in range(len(catalog))]

for i in tqdm(range(len(catalog))):
    corvmodel = corv.models.make_koester_model(resolution = 1)
    param_res, rv_res, rv_init = corv.fit.fit_corv(np.array(catalog['wd_wl'][i]), np.array(catalog['wd_fl'][i]), np.array(catalog['wd_ivar'][i]), corvmodel)
    
    if rv_res.params['teff'].stderr == None:
        rv_res.params['teff'].stderr = 1e-6
    if rv_res.params['logg'].stderr == None:
        rv_res.params['logg'].stderr = 1e-6
    if rv_res.params['RV'].stderr == None:
        rv_res.params['RV'].stderr = 1e-6
    
    f = corv.utils.lineplot(np.array(catalog['wd_wl'][i]), np.array(catalog['wd_fl'][i]), np.array(catalog['wd_ivar'][i]), corvmodel, rv_res.params, gap = 0.3, printparams = True,
             figsize = (6, 5))
    
    f.savefig('spectra/corvfits/corvfit{}.jpg'.format(i))
            
    rvs.append(rv_res.params['RV'].value)
    e_rvs.append(rv_res.params['RV'].stderr)
    redchi.append(rv_res.redchi)


        
#catalog.remove_rows(drops)
catalog['rv'] = rvs
catalog['e_rv'] = e_rvs
catalog['e_rv'][catalog['e_rv'] == None] = 1e-6
catalog['rechi'] = redchi

In [ ]:
print(len(catalog))
print(len(rvs))
catalog = catalog[catalog['rechi'] < 2.5]
catalog = catalog[(catalog['rv'] / catalog['e_rv']) > 5]
catalog = catalog[catalog['rv'] < 1000]

In [ ]:
catalog['e_rv'].pprint(max_lines=-1)

In [ ]:
catalog['e_rv'] = np.array(catalog['e_rv']).astype(float)

In [ ]:
plt.figure(figsize=(25,16))


has_erv = []
for i in range(len(catalog)):
    if catalog['e_rv'][i] > 1e-4:
        has_erv.append([i, np.mean(catalog['wd_ivar'][i])])
        
has_erv = np.array(has_erv).T

num = np.array([i for i in range(len(catalog))])
mean_ivar = np.array([np.mean(catalog['wd_ivar'][i]) for i in range(len(catalog))])

'''
===============================================
                Plot One
===============================================
'''

i = [0,5]

plt.subplot(3,2,1)
plt.scatter(num, mean_ivar, label = 'No Error Reported')
plt.scatter(has_erv[0], has_erv[1], label = 'Error Reported')

#plt.hlines(1, 0, 40)
plt.grid()
plt.ylabel(r'Mean ivar')
plt.xlabel(r'Spectrum Number')
#plt.title('White Dwarf Spectrum')
plt.legend()

plt.subplot(3,2,2)

plt.scatter(num, mean_ivar, label = 'No Error Reported')
plt.scatter(has_erv[0], has_erv[1], label = 'Error Reported')

#plt.hlines(1, 0, 40)
plt.grid()
plt.ylabel(r'Mean ivar')
plt.xlabel(r'Spectrum Number')
plt.legend()

'''
===============================================
    Plot Two
===============================================
'''

plt.subplot(3,2,3)

plt.plot(catalog['wd_wl'][i[0]], catalog['wd_fl'][i[0]])
plt.text(0.05, 0.14, 'Redchi: {}'.format(round(catalog['rechi'][i[0]], 3)), fontsize=12,
                transform = plt.gca().transAxes)

plt.grid()
plt.ylabel(r'ivar')
plt.xlabel(r'Wavelength')
plt.title('Spectrum Number {} (No Error Reported)'.format(i[0]))

plt.subplot(3,2,4)
color = plt.cm.coolwarm(catalog['wd_ivar'][i[0]])
plt.scatter(catalog['wd_wl'][i[0]], catalog['wd_fl'][i[0]], c=catalog['wd_ivar'][i[0]], cmap = 'coolwarm', s=10)

plt.grid()
plt.ylabel(r'Flux')
plt.xlabel(r'Wavelength')
plt.title('Spectrum Number {} (No Error Reported)'.format(i[0]))
plt.colorbar(label='ivar')

'''
===============================================
    Plot Three
===============================================
'''

plt.subplot(3,2,5)

plt.plot(catalog['wd_wl'][i[1]], catalog['wd_fl'][i[1]])
plt.text(0.05, 0.14, 'Redchi: {}'.format(round(catalog['rechi'][i[1]], 3)), fontsize=12,
                transform = plt.gca().transAxes)


plt.grid()
plt.ylabel(r'ivar')
plt.xlabel(r'Wavelength')
plt.title('Spectrum Number {} (Error Reported)'.format(i[1]))

plt.subplot(3,2,6)
#color = plt.cm.coolwarm(catalog['wd_ivar'][i])
plt.scatter(catalog['wd_wl'][i[1]], catalog['wd_fl'][i[1]], c=catalog['wd_ivar'][i[1]], cmap = 'coolwarm', s=10)

plt.grid()
plt.ylabel(r'Flux')
plt.xlabel(r'Wavelength')
plt.title('Spectrum Number {} (Error Reported)'.format(i[1]))
plt.legend()
plt.colorbar(label='ivar')


plt.show()

In [ ]:
print(catalog[['rv','e_rv']])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15, 9))
#axes.grid()
axes.hist(catalog['ms_rv'], bins = 15, alpha=0.5, color='black', label='Main sequence')
#axes.hist(catalog['rv'], bins = 15, alpha=0.5, color='red', label='White dwarf')
ymin, ymax = axes.get_ylim()
axes.vlines(np.mean(catalog['ms_rv']), ymin, ymax, linestyles='dashed')
#axes.vlines(np.mean(catalog['rv']), ymin, ymax, linestyles='dashed', color='red')

#axes[0].set_title('Gravitational Redshift', fontsize=20)
axes.set_xlabel(r'$RV [km/s]$', fontsize=18)
axes.legend()

In [ ]:
catalog['g_redshift'] = catalog['rv'] - catalog['ms_rv']
catalog['eg_redshift'] = catalog['ms_erv'] + catalog['e_rv']

#catalog = catalog[catalog['g_redshift'] > -100]
#catalog = catalog[catalog['g_redshift'] < 100]

print(np.mean(catalog['g_redshift']))
print(len(catalog))


plt.figure(figsize=(10,10))
plt.grid()
plt.hist(catalog['g_redshift'], bins = 20, histtype='step', color='black')
ymin, ymax = plt.ylim()
plt.vlines(np.mean(catalog['g_redshift']), ymin, ymax, linestyles='dashed')
plt.title('Gravitational Redshift', fontsize=20)
plt.xlabel(r'$RV_{MS} - RV_{WD}$', fontsize=18)

In [ ]:
for col in catalog.columns:
    if catalog[col].dtype == object: print(col)

In [ ]:
catalog.remove_columns(['wd_wl', 'wd_fl', 'wd_ivar'])

catalog.write('data/02_wd_rvs.fits', format='fits', overwrite=True)